In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np
# import julian as jl

import matplotlib.pyplot as plt
import matplotlib as mpl

import lxml.html
import requests
import datetime as dt

#from sec_edgar_downloader import Downloader
#from edgar import Company, XBRL, XBRLElement
from pandas import Series, DataFrame
from pandas.plotting import register_matplotlib_converters

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA

register_matplotlib_converters()

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=6d0fa3bd07c41bb64619bc22620e1ed71547f7550909a62e37e0b51c5374ad67
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
PAST_FUTURE_FACTOR = 4
FUTURE_PRED_DAYS = 365

tdFuturePrediction = dt.timedelta(days=FUTURE_PRED_DAYS)
tdPast = tdFuturePrediction * PAST_FUTURE_FACTOR
gIndex = 1

dtStartPeriod = dt.datetime(year=2008,month=1,day=1)
dtDataPeriod = dt.timedelta(days=365)
#dtEndPeriod = dtStartPeriod + dtDataPeriod
STOCKS_SCAN_PERIOD_YEARS = 10
STOCKS_SCAN_INTERVAL = '1mo' # Note options in function
PRED_ROLLING_MEAN_FROM_PAST_VAL_NUM = 12 # should be comparable with FUTURE_PRED_DAYS in units of STOCKS_SCAN_INTERVAL

STOCKS_SCAN_AVG_WINDOW = 3 * 1 # roling window in interval units
dfStockValuesLabled = DataFrame(columns=['StockName', 'StartTime', 'Period', 'StockAtStart', 'StockAtEnd','LableTime', 'LableValue'])

In [ ]:
tickers = pd.read_csv('Nasdaq100_TickersOnly.csv')

In [ ]:
# for tickerName in tickers['Ticker']:
tickerData = yf.Ticker(tickerName)
# dfStockData = tickerData.history(period=str(STOCKS_SCAN_PERIOD_YEARS)+"y", start=dtStartPeriod,interval=STOCKS_SCAN_INTERVAL)
dfStockData = yf.download(tickers=list(tickers['Ticker']),end='2019-01-01', interval='1d', start='2016-01-01')
# dfStockData = dfStockData.dropna()
print(dfStockData)

[*********************100%***********************]  103 of 103 completed

2 Failed downloads:
- FOX: Data doesn't exist for startDate = 1451606400, endDate = 1546300800
- FOXA: Data doesn't exist for startDate = 1451606400, endDate = 1546300800
            Adj Close                         ...      Volume                      
                  AAL       AAPL        ADBE  ...        WLTW        XEL       XLNX
Date                                          ...                                  
2016-01-04  39.095455  24.443037   91.970001  ...  22046900.0  2819300.0  3472200.0
2016-01-05  38.722763  23.830513   92.339996  ...   2163600.0  2141200.0  3895800.0
2016-01-06  39.401257  23.364161   91.019997  ...   2386400.0  5191900.0  2748000.0
2016-01-07  38.655853  22.378082   89.110001  ...   2489500.0  5212200.0  3939000.0
2016-01-08  38.579407  22.496408   87.849998  ...   2006300.0  3005400.0  4067000.0
...               ...        ...         ...  ...         ...        ...        ...

In [ ]:
dfStockData['Close', 'AMZN'][0:701]

Date
2016-01-04     636.989990
2016-01-05     633.789978
2016-01-06     632.650024
2016-01-07     607.940002
2016-01-08     607.049988
                 ...     
2018-10-05    1889.650024
2018-10-08    1864.420044
2018-10-09    1870.319946
2018-10-10    1755.250000
2018-10-11    1719.359985
Name: (Close, AMZN), Length: 701, dtype: float64

In [ ]:
# MWUA: the multiplicative weights update algorithm
NUM_STOCKS = len(tickers['Ticker'])
def get_reward(t):
  reward = np.zeros(NUM_STOCKS)
  for i, tickerName in enumerate(tickers['Ticker']):
    reward[i] = ((dfStockData['Close',tickerName][t]) - (dfStockData['Close',tickerName][t-1]))/(dfStockData['Close',tickerName][t-1])
  reward[np.isnan(reward)] = 0
  return reward
def MWUA(eta, numRounds):
   weights_ = np.ones(NUM_STOCKS)
   M = 1
   for t in range(1, numRounds):
      weights = weights_ / (np.sum(np.abs(weights_)))
      reward = get_reward(t)
      gain = 1 + (weights * reward).sum()
      M *= gain
      print(M)
      weights_ = weights_ * np.exp(eta * reward)
      # print(weights[7], dfStockData['Close','AMZN'][t] - dfStockData['Open','AMZN'][t])

In [ ]:
MWUA(20, dfStockData.shape[0])

0.9986205504748897
1.0018351429725734
0.973499427619967
0.9495558960677448
0.9510434266074119
0.9642929709524738
0.8815739843882624
0.8922781833842749
0.8721378011001882
0.8840976320214767
0.8709494106098657
0.8636842498622687
0.8805035086974972
0.8827350006426228
0.8948582818729007
0.8799372881626172
0.8834818735200185
0.908519803186179
0.911555181092411
0.9077504411476371
0.8928585979586245
0.9007326935504643
0.9187456326515622
0.9036659154163063
0.8919376828864921
0.869067508982874
0.8474599125062423
0.8462669746196978
0.8580498261517567
0.8607230327425899
0.8651677759862929
0.8602985565188428
0.8731506302964332
0.8584048320385873
0.8654679266436636
0.8718184835611437
0.8652008279210809
0.8611900985652122
0.8801212221127991
0.8764386643701888
0.8883745158174124
0.8913557178482805
0.9062956209752187
0.9145779979651545
0.9129752691003439
0.927932180644275
0.927932144756316
0.9257947531859494
0.9217870219512413
0.929268144071673
0.9327432731533425
0.9309456566014824
0.9326211589051002


In [ ]:
sum = 0
for t in range(700):
  print(get_reward(t))
  sum += get_reward(t)
print(sum)